### train 데이터를 annotation과 함께 출력

In [9]:
import cv2
import random
from detectron2.utils.visualizer import Visualizer
from detectron2.data.datasets import register_coco_instances
from detectron2.data import MetadataCatalog, DatasetCatalog
import matplotlib.pyplot as plt

In [10]:
# 리소스
coco_dataset_train = 'coco_trash_train'

# 경로
path_dataset = '/data/ephemeral/home/dataset/'

data_size = 10
scale = 0.6

In [ ]:
if coco_dataset_train not in DatasetCatalog.list():
    register_coco_instances(coco_dataset_train, {}, path_dataset + 'train.json', path_dataset)

dataset_dicts = DatasetCatalog.get(coco_dataset_train)
metadata = MetadataCatalog.get(coco_dataset_train)

for data in random.sample(dataset_dicts, data_size):
    img = cv2.imread(data["file_name"])
    
    visualizer = Visualizer(img[:, :, ::-1], metadata=metadata, scale=scale)
    vis = visualizer.draw_dataset_dict(data)
    
    plt.figure(figsize=(10, 10))
    plt.imshow(vis.get_image()[:, :, ::-1])
    plt.axis("off")
    plt.show()

### 이전 train 결과 보기

In [ ]:
from detectron2.utils.visualizer import Visualizer, ColorMode
from detectron2 import model_zoo
import cv2
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.data import MetadataCatalog
import matplotlib.pyplot as plt

im = cv2.imread("/data/ephemeral/home/dataset/test/0000.jpg")

# 예측
cfg = get_cfg()
cfg.MODEL.DEVICE = "cuda"
cfg.DATASETS.TRAIN = ("coco_trash_train",) 
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml"))
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url('COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml')
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.05

predictor = DefaultPredictor(cfg)
outputs = predictor(im)

# 예측된 바운딩 박스 시각화
v = Visualizer(im[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TEST[0]), scale=1.2)
out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
plt.imshow(out.get_image()[:, :, ::-1])
plt.show()